In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline

class clf:
  def __init__(self):
    self.vectorizer = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1, 1), stop_words = 'english', binary=True)
    self.model = LogisticRegression(n_jobs=1, C=1e5)
  
  def train(self, X_train, y_train, X_dev=None, y_dev=None):
    train_vectors = self.vectorizer.fit_transform(X_train)
    self.model.fit(train_vectors, y_train)
    # uncomment below to use devevlopment data
    #val_preds = self.model.predict(vectorizer.transform(X_dev))

  def tokenise(self, text):
    idx = self.vectorizer.transform([text]).toarray()[0]
    return np.array(self.vectorizer.get_feature_names())[idx==1]    

  def predict(self, x):
    return self.model.predict(self.vectorizer.transform(x))
  

In [ ]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
!gdown --id 1gSdQCIqVZTb8oGrsBL0m8zcJCW9I3c5x
import pandas as pd
cctk = pd.read_csv("cctk.train.csv.zip") 

test_size = 10000

sample = cctk.sample(test_size*10, random_state=42)
sample["label"] = sample.target.apply(round)
train, test = train_test_split(sample, test_size=test_size)

X_train, X_val, y_train, y_val = train_test_split(train.comment_text, train.label, test_size=test_size, random_state=40)
classifier = clf()
classifier.train(X_train, y_train)
predictions = classifier.predict(X_val)

accuracy = accuracy_score(y_val, predictions)
precision = precision_score(y_val, predictions, average='weighted')
recall = recall_score(y_val, predictions, average='weighted')
f1 = f1_score(y_val, predictions, average='weighted')
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))
print(f"ROC: {roc_auc_score(y_val, predictions)}")

In [ ]:
import numpy as np
import InputErasure as IE

# initialise the input erasure method
input_mask = IE.Mask(classifier)

# choose a correctly predicted a toxic example
show_cases = X_val.iloc[np.where((y_val==1)&(predictions==1))]
case = show_cases.sample(1).to_list()[0]

# apply input erasure
prediction, tweaks_pd, mitigated_text = iner.erase_n_score(case, True)

# show
print(f"THE TEXT:\n{case}\n")
print(f"THE Mitigated TEXT:\n{mitigated_text}")
print ('Explanation for class %s' % iner.class_names[1])
tweaks_pd.head()